# Documentation

## Objective(s)
1. Market Basket Analysis on Preprocessed Data Outputs Optimised for Train Stalling Events

    
## Change Log
1. NA
    


# Initialisation

### Adjust Display Theme of Jupyter Notebook
Optional Step

Key Hotkeys:
1. Run cell: ctrl + enter
2. Delete Cell: DD
3. Undo Delete Cell: Z
4. Cut Cell: X
5. Copy Cell: C
6. Insert Cell Above: A
7. Insert Cell Below: B
8. Convert Cell to Code: Y
9. Convert Cell to Markdown: M

In [1]:
# install jupyterthemes
#!pip install jupyterthemes

# upgrade to latest version
#!pip install --upgrade jupyterthemes

In [2]:
#import jupyterthemes

In [3]:
# Adjust to Dark Theme
#jt -t chesterish

## Load Libraries

In [4]:
#pip install --user --upgrade pandas


In [5]:
# Install a pip package in the current Jupyter kernel
import sys
#!{sys.executable} -m pip install schedule

In [6]:
# Import libraries
#import modin.pandas as pd
import pandas as pd
import numpy as np
import os
import pathlib
import datetime as dt
import time
import os
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules
#import re

## Set Options

In [7]:
# Enable display of all columns for dataframes with many variables
pd.set_option('display.max_columns', None)

## Set Up Core Directories

In [8]:
# Check current directory location
cwd = os.getcwd()
cwd

'C:\\Users\\cftfda01\\Documents\\SBST Train IAMS Project\\scripts'

In [9]:
# Define root file directory folder where the files are being stored
#os.chdir(cwd + alarmLoc)
os.chdir(os.path.dirname(os.getcwd()) + '\\alarm-event-logs')

# Check current directory location
cwd = os.getcwd()

# Check directory location
cwd

'C:\\Users\\cftfda01\\Documents\\SBST Train IAMS Project\\alarm-event-logs'

In [10]:
# Location of Pre-Processed Files
taggedEvents = '\\testOutput\\main'

# Set variables
prefix = 'taggedTrainEvents' 
batch = 'B001-'

## Access Files to be Processed

In [11]:
# Define root file directory folder where the files are being stored
#os.chdir(cwd + HISeventLoc)
os.chdir(cwd + taggedEvents)
# Check directory location
os.getcwd()

'C:\\Users\\cftfda01\\Documents\\SBST Train IAMS Project\\alarm-event-logs\\testOutput\\main'

In [12]:
# Get the list of all files in directory tree at given path
fileList = list()
for (dirpath, dirnames, filenames) in os.walk(os.getcwd()):
    fileList += [os.path.join(dirpath, file) for file in filenames] # use this if you want to append full URL
    #fileList += filenames
    
# Inspect data
print(len(fileList))
print(fileList)

2
['C:\\Users\\cftfda01\\Documents\\SBST Train IAMS Project\\alarm-event-logs\\testOutput\\main\\Potential Train Events Raw-B0001-alarmsTagged.csv', 'C:\\Users\\cftfda01\\Documents\\SBST Train IAMS Project\\alarm-event-logs\\testOutput\\main\\Potential Train Events Raw-SparseMatrix.pkl']


In [13]:
inputFile = fileList[0]
inputFile

'C:\\Users\\cftfda01\\Documents\\SBST Train IAMS Project\\alarm-event-logs\\testOutput\\main\\Potential Train Events Raw-B0001-alarmsTagged.csv'

# Process Log Files

### Load Sparse Dataset

In [14]:
BYTES_TO_MB_DIV = 0.000001
def print_memory_usage_of_data_frame(df):
    mem = round(df.memory_usage().sum() * BYTES_TO_MB_DIV, 3) 
    print("Memory usage is " + str(mem) + " MB")
    

In [15]:
import pickle
  
# Open the file in binary mode
with open('Potential Train Events Raw-SparseMatrix.pkl', 'rb') as file:
      
    # Call load method to deserialze
    myvar = pickle.load(file)
    print(myvar)

        BLANK - BLANK - N/B Platform - 11 - PSD Emergency Control Status - CLOSE ACTIVATED  \
0                                                       0                                    
1                                                       0                                    
2                                                       0                                    
3                                                       0                                    
4                                                       0                                    
...                                                   ...                                    
329196                                                  0                                    
329197                                                  0                                    
329198                                                  0                                    
329199                                                  0   

In [16]:
display(myvar.dtypes)
print_memory_usage_of_data_frame(myvar)

BLANK - BLANK - N/B Platform - 11 - PSD Emergency Control Status - CLOSE ACTIVATED    Sparse[uint8, 0]
BLANK - BLANK - N/B Platform - 11 - PSD Emergency Control Status - NOT ACTIVATED      Sparse[uint8, 0]
BLANK - BLANK - N/B Platform - 11 - PSD Emergency Control Status - OPEN ACTIVATED     Sparse[uint8, 0]
BLANK - BLANK - N/B Platform - 11 - PSD Service Status - FAULTY                       Sparse[uint8, 0]
BLANK - BLANK - N/B Platform - 11 - PSD Service Status - Invalid                      Sparse[uint8, 0]
                                                                                            ...       
TT - BLANK - SUBLOCATION Test Track - 10 - Test Track Key Switch Status - KEY OUT     Sparse[uint8, 0]
TT - BLANK - Test Track - 10 - Test Track Control Selection Status - ISCS NORMAL      Sparse[uint8, 0]
TT - BLANK - Test Track - 10 - Test Track Control Selection Status - TTCU NORMAL      Sparse[uint8, 0]
TT - BLANK - Test Track - 10 - Test Track Key Switch Status - KEY IN     

Memory usage is 2770.227 MB


In [17]:
data_one_hot_sparse = myvar

## Perform Market Basket Analysis

In [18]:
# Apply Apriori Algorithm to Find Requently Occuring EVENT_KEY combos
comboLen_cutoff = 3
freq_EVENT_KEY = apriori(data_one_hot_sparse, min_support = 0.01, low_memory=True, max_len = comboLen_cutoff,
                         use_colnames = True).sort_values('support', ascending = False).reset_index(drop = True)
freq_EVENT_KEY["length"] = freq_EVENT_KEY["itemsets"].apply(lambda x: len(x))

# Inspect data
freq_EVENT_KEY

,support,itemsets,length
0,0.705870,(EMU - XXXXXXXX - Train - 12 - Train Departure...,1
1,0.703792,(EMU - XXXXXXXX - Train - 12 - Train Arrival R...,1
2,0.686170,(EMU - XXXXXXXX - Train - 12 - Train Departure...,2
3,0.244337,(EMU - XXXXXXXX - Train - 12 - Train Hold - NO...,1
4,0.236427,(EMU - XXXXXXXX - Train - 12 - Train Arrival R...,2
...,...,...,...
648,0.010033,(EMU - XXXXXXXX - Train Car - 12 - DT Exit Lig...,2
649,0.010030,(SCS - RTU - ISCS RTU - 51 - PAS Controller Li...,2
650,0.010018,(EMU - XXXXXXXX - Train - 12 - Train Departure...,3
651,0.010015,(EMU - XXXXXXXX - Train Car - 12 - DT Exit Lig...,2


In [19]:
# Inspect data
freq_EVENT_KEY.describe()

,support,length
count,653.000000,653.000000
mean,0.035178,2.486983
std,0.058839,0.656655
min,0.010006,1.000000
25%,0.012457,2.000000
50%,0.017360,3.000000
75%,0.031333,3.000000
max,0.705870,3.000000


In [20]:
# Finding Associations with Frequently Occuring Events
assoc_EVENT_KEY_rules = association_rules(freq_EVENT_KEY, metric = "lift", 
                                                min_threshold = 1).sort_values('lift', ascending = False).reset_index(drop = True)

# Inspect data
assoc_EVENT_KEY_rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(SCS - RTU - ISCS RTU - 51 - ESC Controller Li...,(SCS - RTU - ISCS RTU - 51 - PLC Controller Li...,0.011382,0.010559,0.010070,0.884708,83.787883,0.009950,8.582027
1,(SCS - RTU - ISCS RTU - 51 - PLC Controller Li...,(SCS - RTU - ISCS RTU - 51 - ESC Controller Li...,0.010559,0.011382,0.010070,0.953682,83.787883,0.009950,21.344322
2,(SCS - RTU - ISCS RTU - 51 - ESC Controller Li...,(SCS - RTU - ISCS RTU - 51 - PLC Controller Li...,0.011382,0.011160,0.010565,0.928209,83.170225,0.010438,13.773911
3,(SCS - RTU - ISCS RTU - 51 - PLC Controller Li...,(SCS - RTU - ISCS RTU - 51 - ESC Controller Li...,0.011160,0.011382,0.010565,0.946652,83.170225,0.010438,18.531542
4,(SCS - RTU - ISCS RTU - 51 - PLC Controller Li...,(SCS - RTU - ISCS RTU - 51 - PAS Controller Li...,0.010772,0.011355,0.010158,0.943034,83.051864,0.010036,17.355129
...,...,...,...,...,...,...,...,...,...
2629,(EMU - XXXXXXXX - Train Car - 12 - DT ATO/ATP ...,(EMU - XXXXXXXX - Train - 12 - Pti Initialisat...,0.135337,0.074590,0.010273,0.075910,1.017696,0.000179,1.001428
2630,(EMU - XXXXXXXX - Train - 12 - Pti Initialisat...,(EMU - XXXXXXXX - Train Car - 12 - DT ATO/ATP ...,0.071625,0.141321,0.010273,0.143433,1.014943,0.000151,1.002465
2631,(EMU - XXXXXXXX - Train Car - 12 - DT ATO/ATP ...,(EMU - XXXXXXXX - Train - 12 - Pti Initialisat...,0.141321,0.071625,0.010273,0.072695,1.014943,0.000151,1.001154
2632,(EMU - XXXXXXXX - Train - 12 - Train Stalled i...,(EMU - XXXXXXXX - Train - 12 - Train Emergency...,0.130495,0.149984,0.019687,0.150865,1.005870,0.000115,1.001037


In [21]:
# Inspect data
assoc_EVENT_KEY_rules.describe()

,antecedent support,consequent support,support,confidence,lift,leverage,conviction
count,2634.000000,2634.000000,2634.000000,2634.000000,2634.000000,2634.000000,2634.000000
mean,0.192387,0.192387,0.029105,0.383598,4.645694,0.012642,inf
std,0.234264,0.234264,0.036640,0.341053,11.854991,0.015046,NaN
min,0.010173,0.010173,0.010006,0.014193,1.005870,0.000115,1.000882
25%,0.038095,0.038095,0.012457,0.092187,1.372807,0.004350,1.033325
50%,0.104820,0.104820,0.017032,0.249917,1.624828,0.008186,1.160895
75%,0.211047,0.211047,0.029641,0.654578,3.201719,0.014693,2.394773
max,0.705870,0.705870,0.686170,1.000000,83.787883,0.189385,inf


In [22]:
# Calculate Zhang's Metric
assoc_EVENT_KEY_rules["Zhang_numer"] = (assoc_EVENT_KEY_rules["support"] - assoc_EVENT_KEY_rules["antecedent support"] * assoc_EVENT_KEY_rules["consequent support"]) 
assoc_EVENT_KEY_rules["Zhang_denom1"] = (assoc_EVENT_KEY_rules["support"] * (1 - assoc_EVENT_KEY_rules["antecedent support"]))
assoc_EVENT_KEY_rules["Zhang_denom2"] = (assoc_EVENT_KEY_rules["antecedent support"] * (assoc_EVENT_KEY_rules["consequent support"] - assoc_EVENT_KEY_rules["support"]))
assoc_EVENT_KEY_rules["Zhang_denom3"] = assoc_EVENT_KEY_rules[["Zhang_denom1", "Zhang_denom2"]].max(axis=1, skipna=None)
assoc_EVENT_KEY_rules["Zhang"] = (assoc_EVENT_KEY_rules["Zhang_numer"] / assoc_EVENT_KEY_rules["Zhang_denom3"]).round(5)

assoc_EVENT_KEY_rules = assoc_EVENT_KEY_rules.drop(["Zhang_numer", 
                                                    "Zhang_denom1", 
                                                    "Zhang_denom2", 
                                                    "Zhang_denom3"], axis = "columns")
# Inspect data
assoc_EVENT_KEY_rules.head()

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,Zhang
0,(SCS - RTU - ISCS RTU - 51 - ESC Controller Li...,(SCS - RTU - ISCS RTU - 51 - PLC Controller Li...,0.011382,0.010559,0.010070,0.884708,83.787883,0.009950,8.582027,0.99944
1,(SCS - RTU - ISCS RTU - 51 - PLC Controller Li...,(SCS - RTU - ISCS RTU - 51 - ESC Controller Li...,0.010559,0.011382,0.010070,0.953682,83.787883,0.009950,21.344322,0.99861
2,(SCS - RTU - ISCS RTU - 51 - ESC Controller Li...,(SCS - RTU - ISCS RTU - 51 - PLC Controller Li...,0.011382,0.011160,0.010565,0.928209,83.170225,0.010438,13.773911,0.99935
3,(SCS - RTU - ISCS RTU - 51 - PLC Controller Li...,(SCS - RTU - ISCS RTU - 51 - ESC Controller Li...,0.011160,0.011382,0.010565,0.946652,83.170225,0.010438,18.531542,0.99913
4,(SCS - RTU - ISCS RTU - 51 - PLC Controller Li...,(SCS - RTU - ISCS RTU - 51 - PAS Controller Li...,0.010772,0.011355,0.010158,0.943034,83.051864,0.010036,17.355129,0.99872


In [23]:
# Inspect data
assoc_EVENT_KEY_rules.describe()

,antecedent support,consequent support,support,confidence,lift,leverage,conviction,Zhang
count,2634.000000,2634.000000,2634.000000,2634.000000,2634.000000,2634.000000,2634.000000,2634.000000
mean,0.192387,0.192387,0.029105,0.383598,4.645694,0.012642,inf,0.608758
std,0.234264,0.234264,0.036640,0.341053,11.854991,0.015046,NaN,0.292196
min,0.010173,0.010173,0.010006,0.014193,1.005870,0.000115,1.000882,0.006710
25%,0.038095,0.038095,0.012457,0.092187,1.372807,0.004350,1.033325,0.296512
50%,0.104820,0.104820,0.017032,0.249917,1.624828,0.008186,1.160895,0.652890
75%,0.211047,0.211047,0.029641,0.654578,3.201719,0.014693,2.394773,0.893415
max,0.705870,0.705870,0.686170,1.000000,83.787883,0.189385,inf,1.000000


In [24]:
# Prune association rules
# Support = probability of rule combo occuring; higher the better
# Confidence = conditional probability of the rule combo the subset items of the rules; higher the better
# Lift = predective confidence of the antecedent leading to the consequent event
# Zhang's Metric = Range of -1 (Perfect Disassociation) to 1 (Perfect Association)
assoc_EVENT_KEY_rules_f = assoc_EVENT_KEY_rules[
                                              #(assoc_EVENT_KEY_rules["antecedent support"] > 0.1) &
                                              #(assoc_EVENT_KEY_rules["support"] > 0.1) &
                                              #(assoc_EVENT_KEY_rules["confidence"] > 0.9) &
                                              #(assoc_EVENT_KEY_rules["lift"] > 8) &
                                              (assoc_EVENT_KEY_rules["Zhang"] >= 0.70)
                                             ].reset_index(drop = True)

# Inspect data
assoc_EVENT_KEY_rules_f.describe()

,antecedent support,consequent support,support,confidence,lift,leverage,conviction,Zhang
count,1215.000000,1215.000000,1215.000000,1215.000000,1215.000000,1215.000000,1215.000000,1215.000000
mean,0.295940,0.063129,0.032531,0.312515,8.229173,0.017696,inf,0.888006
std,0.295945,0.062192,0.043825,0.300661,16.755671,0.018755,NaN,0.079224
min,0.010188,0.010173,0.010018,0.014193,1.263353,0.002346,1.003376,0.700070
25%,0.049608,0.019028,0.012707,0.053615,1.392209,0.008365,1.019011,0.828445
50%,0.141321,0.039578,0.018290,0.195316,3.425242,0.012156,1.174314,0.902920
75%,0.703792,0.069605,0.031826,0.530563,5.818830,0.018742,1.888850,0.951730
max,0.705870,0.705870,0.686170,1.000000,83.787883,0.189385,inf,1.000000


In [25]:
# Inspect data
assoc_EVENT_KEY_rules_f.head()

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,Zhang
0,(SCS - RTU - ISCS RTU - 51 - ESC Controller Li...,(SCS - RTU - ISCS RTU - 51 - PLC Controller Li...,0.011382,0.010559,0.010070,0.884708,83.787883,0.009950,8.582027,0.99944
1,(SCS - RTU - ISCS RTU - 51 - PLC Controller Li...,(SCS - RTU - ISCS RTU - 51 - ESC Controller Li...,0.010559,0.011382,0.010070,0.953682,83.787883,0.009950,21.344322,0.99861
2,(SCS - RTU - ISCS RTU - 51 - ESC Controller Li...,(SCS - RTU - ISCS RTU - 51 - PLC Controller Li...,0.011382,0.011160,0.010565,0.928209,83.170225,0.010438,13.773911,0.99935
3,(SCS - RTU - ISCS RTU - 51 - PLC Controller Li...,(SCS - RTU - ISCS RTU - 51 - ESC Controller Li...,0.011160,0.011382,0.010565,0.946652,83.170225,0.010438,18.531542,0.99913
4,(SCS - RTU - ISCS RTU - 51 - PLC Controller Li...,(SCS - RTU - ISCS RTU - 51 - PAS Controller Li...,0.010772,0.011355,0.010158,0.943034,83.051864,0.010036,17.355129,0.99872


In [26]:
# Inspect data
assoc_EVENT_KEY_rules_f.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1215 entries, 0 to 1214
Data columns (total 10 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   antecedents         1215 non-null   object 
 1   consequents         1215 non-null   object 
 2   antecedent support  1215 non-null   float64
 3   consequent support  1215 non-null   float64
 4   support             1215 non-null   float64
 5   confidence          1215 non-null   float64
 6   lift                1215 non-null   float64
 7   leverage            1215 non-null   float64
 8   conviction          1215 non-null   float64
 9   Zhang               1215 non-null   float64
dtypes: float64(8), object(2)
memory usage: 95.0+ KB


In [27]:
# Create list of target events:
targetEvents = [
                "Train Emergency Brake",
                "Brake Equipment Status",
                "Service Brake Status",
                "DT Emergency Brake From Other",
                "Emergency Brake by ATC",
                "DT All Brakes Applied Relay Status",
                "DT All Brakes Released Relay Status",
                "Service Brake Summary Status",
                "Emergency Brake Reset Request",
                "Train Stalled in Interstation",
                "Train Skip Stop Demand",
                "ATO - S_ATO_DRIV_ - Stopping point overrun",
                "ATO - S_ATO_DRIV_ - Stopping point underrun",
                "Automatic Hold applied due to TrainCar stalled in "
]

In [28]:
# Convert Consequent and Antecedent data from frozenset to List (first) for easy querying (intermediate step)
assoc_EVENT_KEY_rules_f["antecedents"] = [list(x) for x in assoc_EVENT_KEY_rules_f["antecedents"]]
assoc_EVENT_KEY_rules_f["consequents"] = [list(x) for x in assoc_EVENT_KEY_rules_f["consequents"]]

# Convert Consequent and Antecedent data from list to string for easy querying
assoc_EVENT_KEY_rules_f["antecedents"] = assoc_EVENT_KEY_rules_f["antecedents"].apply(lambda x: ' | '.join(x))
assoc_EVENT_KEY_rules_f["consequents"] = assoc_EVENT_KEY_rules_f["consequents"].apply(lambda x: ' | '.join(x))  


In [29]:
print(assoc_EVENT_KEY_rules_f["antecedents"][0])
print(assoc_EVENT_KEY_rules_f["antecedents"][1])

SCS - RTU - ISCS RTU - 51 - ESC Controller Link Status - STANDBY
SCS - RTU - ISCS RTU - 51 - PLC Controller Link Status - STANDBY | SCS - RTU - ISCS RTU - 51 - LIFT Controller Link Status - STANDBY


In [30]:
print(assoc_EVENT_KEY_rules_f["consequents"][0])
print(assoc_EVENT_KEY_rules_f["consequents"][1])

SCS - RTU - ISCS RTU - 51 - PLC Controller Link Status - STANDBY | SCS - RTU - ISCS RTU - 51 - LIFT Controller Link Status - STANDBY
SCS - RTU - ISCS RTU - 51 - ESC Controller Link Status - STANDBY


In [31]:
targetEvents[0]

'Train Emergency Brake'

In [32]:
assoc_EVENT_KEY_rules_f["consequents"][0]

'SCS - RTU - ISCS RTU - 51 - PLC Controller Link Status - STANDBY | SCS - RTU - ISCS RTU - 51 - LIFT Controller Link Status - STANDBY'

In [33]:
# Filter for train related data
assoc_EVENT_KEY_rules_f["Train Stall Related"] = False
try:
    assoc_EVENT_KEY_rules_f.loc[assoc_EVENT_KEY_rules_f["consequents"].str.contains(targetEvents[0]), "Train Stall Related"] = True
except:
    pass
try:
    assoc_EVENT_KEY_rules_f.loc[assoc_EVENT_KEY_rules_f["consequents"].str.contains(targetEvents[1]), "Train Stall Related"] = True
except:
    pass
try:
    assoc_EVENT_KEY_rules_f.loc[assoc_EVENT_KEY_rules_f["consequents"].str.contains(targetEvents[2]), "Train Stall Related"] = True
except:
    pass
try:
    assoc_EVENT_KEY_rules_f.loc[assoc_EVENT_KEY_rules_f["consequents"].str.contains(targetEvents[3]), "Train Stall Related"] = True
except:
    pass
try:
    assoc_EVENT_KEY_rules_f.loc[assoc_EVENT_KEY_rules_f["consequents"].str.contains(targetEvents[4]), "Train Stall Related"] = True
except:
    pass
try:
    assoc_EVENT_KEY_rules_f.loc[assoc_EVENT_KEY_rules_f["consequents"].str.contains(targetEvents[5]), "Train Stall Related"] = True
except:
    pass
try:
    assoc_EVENT_KEY_rules_f.loc[assoc_EVENT_KEY_rules_f["consequents"].str.contains(targetEvents[6]), "Train Stall Related"] = True
except:
    pass
try:
    assoc_EVENT_KEY_rules_f.loc[assoc_EVENT_KEY_rules_f["consequents"].str.contains(targetEvents[7]), "Train Stall Related"] = True
except:
    pass
try:
    assoc_EVENT_KEY_rules_f.loc[assoc_EVENT_KEY_rules_f["consequents"].str.contains(targetEvents[8]), "Train Stall Related"] = True
except:
    pass
try:
    assoc_EVENT_KEY_rules_f.loc[assoc_EVENT_KEY_rules_f["consequents"].str.contains(targetEvents[9]), "Train Stall Related"] = True
except:
    pass
try:
    assoc_EVENT_KEY_rules_f.loc[assoc_EVENT_KEY_rules_f["consequents"].str.contains(targetEvents[10]), "Train Stall Related"] = True
except:
    pass
try:
    assoc_EVENT_KEY_rules_f.loc[assoc_EVENT_KEY_rules_f["consequents"].str.contains(targetEvents[11]), "Train Stall Related"] = True
except:
    pass
try:
    assoc_EVENT_KEY_rules_f.loc[assoc_EVENT_KEY_rules_f["consequents"].str.contains(targetEvents[12]), "Train Stall Related"] = True
except:
    pass
try:
    assoc_EVENT_KEY_rules_f.loc[assoc_EVENT_KEY_rules_f["consequents"].str.contains(targetEvents[13]), "Train Stall Related"] = True
except:
    pass

# Inspect data
assoc_EVENT_KEY_rules_f.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1215 entries, 0 to 1214
Data columns (total 11 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   antecedents          1215 non-null   object 
 1   consequents          1215 non-null   object 
 2   antecedent support   1215 non-null   float64
 3   consequent support   1215 non-null   float64
 4   support              1215 non-null   float64
 5   confidence           1215 non-null   float64
 6   lift                 1215 non-null   float64
 7   leverage             1215 non-null   float64
 8   conviction           1215 non-null   float64
 9   Zhang                1215 non-null   float64
 10  Train Stall Related  1215 non-null   bool   
dtypes: bool(1), float64(8), object(2)
memory usage: 96.2+ KB


In [34]:
# Inspect data
assoc_EVENT_KEY_rules_f.head()

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,Zhang,Train Stall Related
0,SCS - RTU - ISCS RTU - 51 - ESC Controller Lin...,SCS - RTU - ISCS RTU - 51 - PLC Controller Lin...,0.011382,0.010559,0.010070,0.884708,83.787883,0.009950,8.582027,0.99944,False
1,SCS - RTU - ISCS RTU - 51 - PLC Controller Lin...,SCS - RTU - ISCS RTU - 51 - ESC Controller Lin...,0.010559,0.011382,0.010070,0.953682,83.787883,0.009950,21.344322,0.99861,False
2,SCS - RTU - ISCS RTU - 51 - ESC Controller Lin...,SCS - RTU - ISCS RTU - 51 - PLC Controller Lin...,0.011382,0.011160,0.010565,0.928209,83.170225,0.010438,13.773911,0.99935,False
3,SCS - RTU - ISCS RTU - 51 - PLC Controller Lin...,SCS - RTU - ISCS RTU - 51 - ESC Controller Lin...,0.011160,0.011382,0.010565,0.946652,83.170225,0.010438,18.531542,0.99913,False
4,SCS - RTU - ISCS RTU - 51 - PLC Controller Lin...,SCS - RTU - ISCS RTU - 51 - PAS Controller Lin...,0.010772,0.011355,0.010158,0.943034,83.051864,0.010036,17.355129,0.99872,False


In [35]:
assoc_EVENT_KEY_rules_f.loc[assoc_EVENT_KEY_rules_f["Train Stall Related"] == True]

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,Zhang,Train Stall Related
90,EMU - XXXXXXXX - Train - 12 - Train Hold Reque...,EMU - XXXXXXXX - Train Car - 12 - DT ATO/ATP S...,0.062955,0.012467,0.011300,0.179493,14.398001,0.010515,1.203565,0.99306,True
93,EMU - XXXXXXXX - Train - 12 - Train Hold Reque...,EMU - XXXXXXXX - Train - 12 - Train Skip Stop ...,0.062955,0.012044,0.010878,0.172786,14.345898,0.010120,1.194318,0.99280,True
94,EMU - XXXXXXXX - Train Car - 12 - DT ATO/ATP S...,EMU - XXXXXXXX - Train - 12 - Train Skip Stop ...,0.012618,0.069605,0.012576,0.996630,14.318387,0.011698,276.061520,0.94205,True
96,EMU - XXXXXXXX - Train - 12 - Train Arrival Re...,EMU - XXXXXXXX - Train - 12 - Train Skip Stop ...,0.061048,0.069605,0.060774,0.995522,14.302468,0.056525,207.757228,0.99055,True
99,EMU - XXXXXXXX - Train - 12 - Train Hold Reque...,EMU - XXXXXXXX - Train - 12 - Train Skip Stop ...,0.062955,0.067497,0.060774,0.965356,14.302255,0.056525,26.916615,0.99257,True
...,...,...,...,...,...,...,...,...,...,...,...
1169,EMU - XXXXXXXX - Train - 12 - Train Arrival Re...,EMU - XXXXXXXX - Train Car - 12 - DT Exit Ligh...,0.703792,0.011580,0.010875,0.015452,1.334399,0.002725,1.003933,0.84602,True
1177,EMU - XXXXXXXX - Train - 12 - Train Departure ...,EMU - XXXXXXXX - Train - 12 - Train Stalled in...,0.705870,0.011291,0.010608,0.015028,1.330936,0.002638,1.003794,0.84537,True
1179,EMU - XXXXXXXX - Train - 12 - Train Departure ...,EMU - XXXXXXXX - Train Car - 12 - DT Exit Ligh...,0.705870,0.011580,0.010866,0.015393,1.329357,0.002692,1.003873,0.84234,True
1185,EMU - XXXXXXXX - Train - 12 - Train Arrival Re...,EMU - XXXXXXXX - Train - 12 - Train Stalled in...,0.703792,0.014417,0.013469,0.019138,1.327467,0.003323,1.004813,0.83281,True


In [36]:
cwd

'C:\\Users\\cftfda01\\Documents\\SBST Train IAMS Project\\alarm-event-logs'

In [37]:
os.path.basename(inputFile)

'Potential Train Events Raw-B0001-alarmsTagged.csv'

In [38]:
# Define Run No. 
runNum =str(2)+"-"
runNum

winSize = "10s"

In [39]:
# Save File
assoc_EVENT_KEY_rules_f.to_csv('../associationRules/' + prefix + '-rules-' + batch + winSize + '-' + runNum + os.path.basename(inputFile) +'.csv', index=False)
print(os.path.basename(inputFile) + " saved")


Potential Train Events Raw-B0001-alarmsTagged.csv saved
